In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json
import math
import drawsvg

In [3]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

import xylo.just

import xylo.resonators as r

In [4]:
layout = xylo.just.diamond_11_partch

In [29]:
c = r.Config(radius = (0.050-0.0016*2)/2)
c

Config(temp=20, correction=0.61, radius=0.0234)

In [30]:
slack = 0.05
tube_lengths = { k: r.length_of_freq(f, c) + slack for k,f in layout.get_freqs().items() }

In [31]:
sum([v for v in tube_lengths.values() if v > 0.00])

5.705992675644116

In [32]:
tube_lengths

{'A1': 0.07552361394742207,
 'B1': 0.08436752815796031,
 'B2': 0.08547301743427758,
 'C1': 0.09826510763166327,
 'C2': 0.09652791019745038,
 'C3': 0.09542242092113311,
 'D1': 0.10868829223694046,
 'D2': 0.11389988453957908,
 'D3': 0.10868829223694046,
 'D4': 0.10537182440798862,
 'E1': 0.12328075068432857,
 'E2': 0.12692886529617559,
 'E3': 0.12953466144749487,
 'E4': 0.12084867427643055,
 'E5': 0.12527063138169967,
 'F1': 0.14516943835541068,
 'F2': 0.14516943835541068,
 'F3': 0.14516943835541068,
 'F4': 0.14516943835541068,
 'F5': 0.14516943835541068,
 'F6': 0.14516943835541068,
 'G1': 0.17253029794426336,
 'G2': 0.16705812602649284,
 'G3': 0.1634100114146458,
 'G4': 0.17643899217124231,
 'G5': 0.16949020243439084,
 'H1': 0.19989115753311604,
 'H2': 0.188946813697575,
 'H3': 0.19989115753311604,
 'H4': 0.20770854598707394,
 'I1': 0.22725201712196869,
 'I2': 0.23272418903973924,
 'I3': 0.2363723036515863,
 'J1': 0.2819737362996741,
 'J2': 0.27650156438190354,
 'K1': 0.3366954554773793

In [33]:
for k, v in tube_lengths.items():
  if v >= 0.08:
    print(f"{k}: {v}m ({layout.get_freqs()[k]}Hz)")

B1: 0.08436752815796031m (1764.0Hz)
B2: 0.08547301743427758m (1724.8000000000002Hz)
C1: 0.09826510763166327m (1372.0Hz)
C2: 0.09652791019745038m (1411.2Hz)
C3: 0.09542242092113311m (1437.3333333333333Hz)
D1: 0.10868829223694046m (1176.0Hz)
D2: 0.11389988453957908m (1097.6Hz)
D3: 0.10868829223694046m (1176.0Hz)
D4: 0.10537182440798862m (1232.0Hz)
E1: 0.12328075068432857m (980.0Hz)
E2: 0.12692886529617559m (940.8Hz)
E3: 0.12953466144749487m (914.6666666666667Hz)
E4: 0.12084867427643055m (1008.0000000000001Hz)
E5: 0.12527063138169967m (958.2222222222223Hz)
F1: 0.14516943835541068m (784.0Hz)
F2: 0.14516943835541068m (784.0Hz)
F3: 0.14516943835541068m (784.0Hz)
F4: 0.14516943835541068m (784.0Hz)
F5: 0.14516943835541068m (784.0Hz)
F6: 0.14516943835541068m (784.0Hz)
G1: 0.17253029794426336m (627.2Hz)
G2: 0.16705812602649284m (653.3333333333334Hz)
G3: 0.1634100114146458m (672.0Hz)
G4: 0.17643899217124231m (609.7777777777778Hz)
G5: 0.16949020243439084m (641.4545454545455Hz)
H1: 0.19989115753311

In [34]:
def mkres(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mkres(tube_lengths, 'dia-resonator-lengths')

A1 0.07552361394742207
B1 0.08436752815796031
B2 0.08547301743427758
C1 0.09826510763166327
C2 0.09652791019745038
C3 0.09542242092113311
D1 0.10868829223694046
D2 0.11389988453957908
D3 0.10868829223694046
D4 0.10537182440798862
E1 0.12328075068432857
E2 0.12692886529617559
E3 0.12953466144749487
E4 0.12084867427643055
E5 0.12527063138169967
F1 0.14516943835541068
F2 0.14516943835541068
F3 0.14516943835541068
F4 0.14516943835541068
F5 0.14516943835541068
F6 0.14516943835541068
G1 0.17253029794426336
G2 0.16705812602649284
G3 0.1634100114146458
G4 0.17643899217124231
G5 0.16949020243439084
H1 0.19989115753311604
H2 0.188946813697575
H3 0.19989115753311604
H4 0.20770854598707394
I1 0.22725201712196869
I2 0.23272418903973924
I3 0.2363723036515863
J1 0.2819737362996741
J2 0.27650156438190354
K1 0.33669545547737934
